# Top Pages (Last 7 Days)

This notebook queries GA4 export data in BigQuery and shows which pages received the most traffic in the last 7 days.

### Quick start (beginner-friendly)
1. Run the **Setup (run once)** cell.
2. In **Parameters**, choose `TOP_N` (how many pages to list).
3. Run the remaining cells from top to bottom.

### Links
- GitHub repo: [github.com/aidanm-lla/lla-data](https://github.com/aidanm-lla/lla-data)
- Open this notebook in Colab: [Top Pages (Last 7 Days)](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/top_pages_last_7_days.ipynb)

### Other notebooks
- [Search Contribution Overview](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/01_search_contribution_overview.ipynb)
- [Top Pages Search Performance](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/02_top_pages_search_performance.ipynb)
- [Query Drivers by Page](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/03_query_drivers_by_page.ipynb)
- [Opportunity Watchlist](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/04_opportunity_watchlist.ipynb)
- [Traffic Source Quality](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/traffic_sources.ipynb)
- [Time Patterns for Crisis-Related Pages](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/time_patterns.ipynb)
- [Crisis Support Funnel](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/crisis_funnel.ipynb)
- [Analysis Template](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/templates/analysis_template.ipynb)

In [ ]:
#@title Setup (run once)
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    if not os.path.exists("lla-data"):
        !git clone -q https://github.com/aidoanto/lla-data.git
    repo = os.path.abspath("lla-data")
    if repo not in sys.path:
        sys.path.insert(0, repo)
    !pip install -q db-dtypes google-cloud-bigquery kaleido plotly
else:
    for p in ("..", "../.."):
        ap = os.path.abspath(p)
        if ap not in sys.path:
            sys.path.insert(0, ap)

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()

In [ ]:
#@title Parameters
TOP_N = 15 #@param {type:"integer"}
DAYS_BACK = 7

In [ ]:
query = f"""
SELECT
  COALESCE((
    SELECT ep.value.string_value
    FROM UNNEST(event_params) ep
    WHERE ep.key = 'page_location'
  ), '(unknown)') AS page_location,
  COUNT(*) AS page_views
FROM `{config.PROJECT_ID}.{config.GA4_DATASET}.events_*`
WHERE event_name = 'page_view'
  AND _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL {DAYS_BACK} DAY))
  AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
GROUP BY page_location
ORDER BY page_views DESC
LIMIT {TOP_N}
"""

df = run_query(client, query)
df

In [ ]:
df["page_path"] = lifeline_theme.strip_domain(df["page_location"])

fig = px.bar(
    df.sort_values("page_views"),          # ascending so top pages appear at the top
    x="page_views",
    y="page_path",
    orientation="h",
    title="Top Pages by Page Views (Last 7 Days)",
    labels={"page_path": "Page", "page_views": "Page Views"},
    template="lifeline",
)
# Wide left margin to accommodate full path labels
fig.update_layout(height=550, yaxis_title=None, margin=dict(l=260))
lifeline_theme.add_lifeline_logo(fig)
fig.show()

If you get an auth error, run this once in terminal:

```bash
gcloud auth application-default login
```